In [2]:
import fastf1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from fastf1.plotting import setup_mpl

setup_mpl(color_scheme='fastf1', misc_mpl_mods=False)  # Use FastF1's default styling

ModuleNotFoundError: No module named 'fastf1'

In [ ]:
# Load the race and practice sessions
race = fastf1.get_session(2021, 'Spain', 'R')
fp1 = fastf1.get_session(2021, 'Spain', 'FP1')
fp2 = fastf1.get_session(2021, 'Spain', 'FP2')
fp3 = fastf1.get_session(2021, 'Spain', 'FP3')

# Load session data
race.load()
fp1.load()
fp2.load()
fp3.load()

In [ ]:
# Get Hamilton & Verstappen's laps
drivers = ['HAM', 'VER']
laps = race.laps[race.laps['Driver'].isin(drivers)].copy()

# Extract tyre compound, age, and lap times
laps['TyreAge'] = laps['LapNumber'] - laps.groupby('Driver')['PitOutTime'].fillna(method='ffill').apply(lambda x: x.dt.total_seconds() // 90)
laps['LapTimeSeconds'] = laps['LapTime'].dt.total_seconds()

# Keep only relevant columns
laps = laps[['LapNumber', 'Driver', 'TyreAge', 'LapTimeSeconds', 'Compound']]
laps.head()

In [ ]:
# Function to get long runs (at least 10 consecutive laps) from a practice session
def get_long_runs(session, driver):
    laps = session.laps.pick_driver(driver).copy()
    laps['LapTimeSeconds'] = laps['LapTime'].dt.total_seconds()

    # Identify stints of 10+ flying laps
    stint_groups = []
    stint = []
    for i in range(len(laps)):
        if i > 0 and laps.iloc[i]['TyreAge'] > laps.iloc[i-1]['TyreAge'] + 1:
            if len(stint) >= 10:
                stint_groups.append(stint)
            stint = []
        stint.append(laps.iloc[i])
    
    if len(stint) >= 10:
        stint_groups.append(stint)
    
    return stint_groups

# Store degradation data
degradation_data = {}

# Loop through both drivers and all practice sessions
for driver in drivers:
    degradation_data[driver] = {}
    
    for session in [fp1, fp2, fp3]:
        for stint in get_long_runs(session, driver):
            df_stint = pd.DataFrame(stint)
            tyre = df_stint.iloc[0]['Compound']

            # Use fastest of first 3 laps as reference
            reference_lap = df_stint.iloc[:3]['LapTimeSeconds'].min()
            df_stint['DeltaToReference'] = df_stint['LapTimeSeconds'] - reference_lap

            # Store degradation data by tyre type
            if tyre not in degradation_data[driver]:
                degradation_data[driver][tyre] = []
            
            degradation_data[driver][tyre].append(df_stint[['TyreAge', 'DeltaToReference']])

In [ ]:
# Function to compute average degradation per tyre type
def get_degradation_model(driver, tyre):
    if tyre not in degradation_data[driver]:
        return None  # No data for this tyre
    
    combined_data = pd.concat(degradation_data[driver][tyre], ignore_index=True)
    
    # Average degradation per tyre age
    avg_degradation = combined_data.groupby('TyreAge')['DeltaToReference'].mean().reset_index()
    
    # Extrapolate to race distance (~66 laps for Spain)
    max_age = 66
    avg_degradation = avg_degradation.set_index('TyreAge').reindex(range(1, max_age + 1)).interpolate().reset_index()
    
    return avg_degradation

# Generate degradation models
degradation_models = {driver: {tyre: get_degradation_model(driver, tyre) for tyre in ['SOFT', 'MEDIUM', 'HARD']} for driver in drivers}

In [ ]:
# Function to simulate race pace based on degradation model
def predict_race_pace(driver):
    pace_prediction = []
    
    current_tyre = None
    reference_lap = None
    
    for _, lap in laps[laps['Driver'] == driver].iterrows():
        if current_tyre != lap['Compound']:
            current_tyre = lap['Compound']
            reference_lap = lap['LapTimeSeconds']
        
        # Get predicted delta based on tyre age
        delta = degradation_models[driver][current_tyre].loc[degradation_models[driver][current_tyre]['TyreAge'] == lap['TyreAge'], 'DeltaToReference'].values
        if len(delta) > 0:
            predicted_lap = reference_lap + delta[0]
        else:
            predicted_lap = reference_lap
        
        pace_prediction.append({'LapNumber': lap['LapNumber'], 'Driver': driver, 'PredictedLapTime': predicted_lap})
    
    return pd.DataFrame(pace_prediction)

# Predict pace for both drivers
predicted_pace = pd.concat([predict_race_pace(driver) for driver in drivers])

In [ ]:
# Merge predictions
gap_data = predicted_pace.pivot(index='LapNumber', columns='Driver', values='PredictedLapTime')
gap_data['GapVER_HAM'] = gap_data['VER'] - gap_data['HAM']

# Plot predicted gap over full race
plt.figure(figsize=(10, 5))
plt.plot(gap_data.index, gap_data['GapVER_HAM'], label="Predicted Gap (VER - HAM)", color='purple')
plt.axvline(x=40, color='red', linestyle='--', label="Lap 40 Mark")
plt.xlabel("Lap Number")
plt.ylabel("Predicted Gap (seconds)")
plt.title("Predicted Gap Between Verstappen & Hamilton Over the Race")
plt.legend()
plt.grid()
plt.show()

# Zoomed-in view from lap 40 onward
plt.figure(figsize=(10, 5))
plt.plot(gap_data.loc[40:].index, gap_data.loc[40:]['GapVER_HAM'], label="Predicted Gap (VER - HAM)", color='purple')
plt.xlabel("Lap Number")
plt.ylabel("Predicted Gap (seconds)")
plt.title("Predicted Gap from Lap 40 Onward")
plt.legend()
plt.grid()
plt.show()